In [1]:
from typing import Tuple, Dict, Any
from preprocessing_utils import *
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch
from tqdm import tqdm
from torch.optim import Adam
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler


device = torch.device("cuda:0" if torch.cuda.is_available() else "mps")

user_item_path = '/Volumes/DeepLearner/Search & Recommendation System/Data/australian_users_items_clean.json'
review_path = '/Volumes/DeepLearner/Search & Recommendation System/Data/steam_reviews_clean.json'

In [2]:
def create_mappings(df, column):
    unique_ids = df[column].unique()
    print(f"Number of unique {column}: {len(unique_ids)}")
    id_to_idx = {id_: idx for idx, id_ in enumerate(unique_ids)}
    return id_to_idx

class SteamDataset(Dataset):
    def __init__(self, user_item_df, review_df, user_mapping, item_mapping):
        self.merged_df = pd.merge(user_item_df, review_df, how='inner', on=['user_id', 'item_id'])
        
        self.merged_df['user_idx'] = self.merged_df['user_id'].map(user_mapping)
        self.merged_df['item_idx'] = self.merged_df['item_id'].map(item_mapping)

        print(f"Missing user_idx: {self.merged_df['user_idx'].isna().sum()}")
        print(f"Missing item_idx: {self.merged_df['item_idx'].isna().sum()}")
        
        
        # TF-IDF Vectorization
        self.tfidf_vectorizer = TfidfVectorizer(max_features=5000)
        self.X_text = self.tfidf_vectorizer.fit_transform(self.merged_df['review']).toarray()
        
        # Scaling playtime
        self.scaler = MinMaxScaler(feature_range=(0, 1))
        self.y = self.scaler.fit_transform(self.merged_df['playtime_forever'].values.reshape(-1, 1)).flatten()
        
        self.user_idxs = self.merged_df['user_idx'].values
        self.item_idxs = self.merged_df['item_idx'].values
    
    def __len__(self):
        return len(self.merged_df)
    
    def __getitem__(self, idx):
        return self.X_text[idx], self.y[idx], self.user_idxs[idx], self.item_idxs[idx]

In [3]:
from sklearn.model_selection import train_test_split

def prepare_data_loaders(user_item_path, review_path, batch_size=32):
    # Load data
    user_item_df = load_json_to_df(user_item_path)
    review_df = load_review_json_to_df(review_path)
    
    # Drop duplicates based on 'user_id' and 'item_id'
    user_item_df.drop_duplicates(subset=['user_id', 'item_id'], inplace=True)
    review_df.drop_duplicates(subset=['user_id', 'item_id'], inplace=True)
    
    # Create an initial dataset to get the merged_df
    initial_dataset = SteamDataset(user_item_df, review_df, {}, {})
    merged_df = initial_dataset.merged_df
    
    # Create mappings based on the merged_df
    user_mapping = create_mappings(merged_df, 'user_id')
    item_mapping = create_mappings(merged_df, 'item_id')

    # Create the final dataset using the new mappings
    full_dataset = SteamDataset(user_item_df, review_df, user_mapping, item_mapping)

    # Split into train, validation, and test sets
    train_size = int(0.7 * len(full_dataset))
    val_size = int(0.15 * len(full_dataset))
    test_size = len(full_dataset) - train_size - val_size

    train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
        full_dataset, [train_size, val_size, test_size])

    # Create DataLoader for each set
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

   # train_loader, val_loader, test_loader, user_mapping, item_mapping, merged_df, tfidf_matrix = prepare_data_loaders(user_item_path, review_path)


    return train_loader, val_loader, test_loader, user_mapping, item_mapping, full_dataset.merged_df, full_dataset.X_text

In [4]:
def get_actual_top_k_games(user_id, merged_df, k=5):
    user_data = merged_df[merged_df['user_idx'] == user_id]
    top_k_actual = user_data.sort_values(by='playtime_forever', ascending=False).head(k)
    return top_k_actual['item_name'].tolist()

In [5]:
def get_predicted_top_k_games(model, user_id, user_mapping, item_mapping, merged_df, tfidf_matrix, k=5):
    model.eval()
    #print("All unique item_idx in merged_df:", merged_df['item_idx'].unique())

    # Convert to tensor and repeat to match the number of items
    user_idx = torch.LongTensor([user_mapping[user_id]] * len(item_mapping)).to(device)
    
    # Make sure to bring the tensor to CPU before converting to NumPy for indexing
    all_item_idxs_cpu = torch.LongTensor(list(item_mapping.values())).cpu().numpy()
    
    # Extract the TF-IDF features for all items, make sure they are on the same device as the model
    tfidf_features = torch.FloatTensor(tfidf_matrix[all_item_idxs_cpu]).to(device)
    
    # Convert all_item_idxs back to a tensor and move to the same device as the model
    all_item_idxs = torch.LongTensor(all_item_idxs_cpu).to(device)

    # Make predictions
    with torch.no_grad():
        predictions = model(user_idx, all_item_idxs, tfidf_features).cpu().numpy().flatten()

    # Extract top-k items
    top_k_indices = predictions.argsort()[-k:][::-1]
    top_k_item_idxs = [list(item_mapping.values())[i] for i in top_k_indices]
    #print("Top K item indices:", top_k_item_idxs)
    #print("All unique item_idx in merged_df:", sorted(merged_df['item_idx'].unique()))
    #print("Top K item indices:", top_k_item_idxs)

    top_k_item_names = [merged_df.loc[merged_df['item_idx'] == idx, 'item_name'].iloc[0] for idx in top_k_item_idxs]

    return top_k_item_names


In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class HybridModel(nn.Module):
    def __init__(self, num_users, num_items, num_text_features, embedding_dim):
        super(HybridModel, self).__init__()
        
        # User and Item Embeddings
        self.user_embedding = nn.Embedding(num_users, embedding_dim)
        self.item_embedding = nn.Embedding(num_items, embedding_dim)
        
        # Text feature layers
        self.text_layers = nn.Sequential(
            nn.Linear(num_text_features, 128),
            nn.ReLU(),
            nn.Linear(128, 32),
            nn.ReLU()
        )
        
        # Combined layers
        self.combined_layers = nn.Sequential(
            nn.Linear(embedding_dim * 2 + 32, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )
        
    def forward(self, user_ids, item_ids, text_features):
        user_embedding = self.user_embedding(user_ids)
        item_embedding = self.item_embedding(item_ids)
        
        text_output = self.text_layers(text_features)
        
        # Concatenate the embeddings and text features
        combined_input = torch.cat([user_embedding, item_embedding, text_output], dim=1)
        
        output = self.combined_layers(combined_input)

        output = torch.sigmoid(output)
        
        return output

In [7]:
train_loader, val_loader, test_loader, user_mapping, item_mapping, merged_df, tfidf_matrix = prepare_data_loaders(user_item_path, review_path)

Missing user_idx: 44186
Missing item_idx: 44186
Number of unique user_id: 19913
Number of unique item_id: 3097
Missing user_idx: 0
Missing item_idx: 0


In [8]:
# Step 1
print("Are there duplicate (user_id, item_id) pairs?", any(merged_df.duplicated(['user_id', 'item_id'])))
print("Missing user_idx:", merged_df['user_idx'].isnull().sum())
print("Missing item_idx:", merged_df['item_idx'].isnull().sum())

# Step 2
print(f"Shape of X_text: {tfidf_matrix.shape[0]}")
print(f"Expected shape: {len(merged_df)}")
print(f"Length of user_mapping: {len(user_mapping)}")
print(f"Length of item_mapping: {len(item_mapping)}")
print(f"Unique user_ids: {merged_df['user_id'].nunique()}")
print(f"Unique item_ids: {merged_df['item_id'].nunique()}")

# Step 4: Validate by printing or manually inspecting
# (You can print sample batches from DataLoader here)

# Step 5: Done in get_predicted_top_k_games function


Are there duplicate (user_id, item_id) pairs? False
Missing user_idx: 0
Missing item_idx: 0
Shape of X_text: 44186
Expected shape: 44186
Length of user_mapping: 19913
Length of item_mapping: 3097
Unique user_ids: 19913
Unique item_ids: 3097


In [9]:
from sklearn.metrics import mean_squared_error
from math import sqrt
# Initialize model, optimizer, and loss function
model = HybridModel(num_users=len(user_mapping), num_items=len(item_mapping), num_text_features=5000, embedding_dim=10)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

# Move model to device
model = model.to(device)

# Number of epochs
n_epochs = 1000

def precision_at_k(y_true, y_pred, k):
    # Sort by predicted score and take top k
    top_k_items = np.argsort(y_pred)[::-1][:k]
    # Count number of true positives in top k
    true_positives = np.sum(y_true[top_k_items])
    return true_positives / k

import random

def predict_for_random_user(model, user_mapping, item_mapping, merged_df, tfidf_matrix, k=5):
    # Randomly select a user ID
    random_user_id = random.choice(list(user_mapping.keys()))
    print(f"Making predictions for random user {random_user_id}")
    
    # Call the existing function to make predictions for this user
    top_k_games = get_predicted_top_k_games(model, random_user_id, user_mapping, item_mapping, merged_df, tfidf_matrix, k)
    return random_user_id, top_k_games

for epoch in range(1, n_epochs + 1):
    
    # Training loop
    model.train()
    train_loss = 0.0
    for batch_data in tqdm(train_loader, desc=f"Training epoch {epoch}"):
        X_batch, y_batch, user_ids, item_ids = batch_data
        
        #print(f"X_batch dtype: {X_batch.dtype}")
       # print(f"y_batch dtype: {y_batch.dtype}")
       # print(f"user_ids dtype: {user_ids.dtype}")
       # print(f"item_ids dtype: {item_ids.dtype}")
        
        # Convert to float32 if needed
        X_batch, y_batch = X_batch.float(), y_batch.float()
        user_ids, item_ids = user_ids.long(), item_ids.long()

        # Move to device
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        user_ids, item_ids = user_ids.to(device), item_ids.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward + backward + optimize
        outputs = model(user_ids, item_ids, X_batch)
        loss = criterion(outputs.squeeze(), y_batch)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
    
    train_loss /= len(train_loader)
    print(f"Epoch {epoch} - Training loss: {train_loss}")
    
    # Validation loop
    model.eval()
    y_true_val = []
    y_pred_val = []
    val_loss = 0.0
    with torch.no_grad():
        for batch_data in tqdm(val_loader, desc=f"Validating epoch {epoch}"):
            X_batch, y_batch, user_ids, item_ids = batch_data
            X_batch, y_batch = X_batch.float(), y_batch.float()
            user_ids, item_ids = user_ids.long(), item_ids.long()

            # Move to device
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            user_ids, item_ids = user_ids.to(device), item_ids.to(device)


            outputs = model(user_ids, item_ids, X_batch)
            loss = criterion(outputs.squeeze(), y_batch)
            
            val_loss += loss.item()
            y_true_val.extend(y_batch.cpu().numpy())
            y_pred_val.extend(outputs.cpu().numpy())
    
    val_loss /= len(val_loader)
    val_rmse = sqrt(mean_squared_error(y_true_val, y_pred_val))
    val_precision_at_k = precision_at_k(np.array(y_true_val), np.array(y_pred_val), k=5)
    
    print(f"Epoch {epoch} - Validation loss: {val_loss}, RMSE: {val_rmse}, Precision@5: {val_precision_at_k}")

    # Display top-k games for a sample user (let's say the first user in our dataset)
    #sample_user_id = list(user_mapping.keys())[0]

    random_user_id, random_top_k_games = predict_for_random_user(model, user_mapping, item_mapping, merged_df, tfidf_matrix)
    print(f"Actual top 5 games for user {random_user_id}: {get_actual_top_k_games(user_mapping[random_user_id], merged_df)}")
    print(f"Predicted top 5 games for user {random_user_id}: {random_top_k_games}")

Training epoch 1: 100%|██████████| 967/967 [00:08<00:00, 120.77it/s]


Epoch 1 - Training loss: 0.005843912543573466


Validating epoch 1: 100%|██████████| 208/208 [00:00<00:00, 314.40it/s]


Epoch 1 - Validation loss: 0.0010694802717457956, RMSE: 0.03262546949544343, Precision@5: 1.5557592269033195e-05
Making predictions for random user The-Burnt_Toast
Actual top 5 games for user The-Burnt_Toast: ['Grand Theft Auto V', 'Warhammer® 40,000™: Dawn of War® II - Chaos Rising™']
Predicted top 5 games for user The-Burnt_Toast: ['Run and Fire', 'Heli Heroes', 'Audiosurf 2', 'Pro Cycling Manager 2013', 'Poof']


Training epoch 2: 100%|██████████| 967/967 [00:07<00:00, 128.99it/s]


...

Epoch 31 - Training loss: 3.60799033908413e-05


Validating epoch 31: 100%|██████████| 208/208 [00:00<00:00, 332.67it/s]


Epoch 31 - Validation loss: 0.0011462355656266692, RMSE: 0.033792754478732145, Precision@5: 1.5557592269033195e-05
Making predictions for random user 76561198064243876
Actual top 5 games for user 76561198064243876: ['Realm of the Mad God']
Predicted top 5 games for user 76561198064243876: ['Project X', 'Rugby Challenge', 'Cult of the Wind', 'Farnham Fables', 'Cognition: An Erica Reed Thriller']


Training epoch 32:  22%|██▏       | 208/967 [00:01<00:05, 132.83it/s]


KeyboardInterrupt: 